# McFarland 2020

In [1]:
#!pip install pertpy[validator]

In [2]:
# Using a local instance here but in practice, we use `laminlabs/pertpy-datasets`
!lamin load laminlabs/pertpy-datasets

→ connected lamindb: laminlabs/pertpy-datasets


In [3]:
import lamindb as ln
import bionty as bt
import wetlab as wl
import findrefs as fr
import pertpy as pt
import scanpy as sc

THIS_INSTANCE = "laminlabs/pertpy-datasets"

→ connected lamindb: laminlabs/pertpy-datasets


In [4]:
ln.track("HIRTYxL3aZc70000")

→ notebook imports: bionty==0.52.0 findrefs==0.1.0 lamindb==0.76.15 pertpy==0.9.4 scanpy==1.10.3 wetlab==0.34.0
→ loaded Transform('HIRTYxL3'), started Run('r4d3DOTu') at 2024-10-30 13:27:28 UTC


In [5]:
adata = pt.dt.mcfarland_2020()
adata.obs.head(3)

,DepMap_ID,cancer,cell_det_rate,cell_line,cell_quality,channel,disease,dose_unit,dose_value,doublet_CL1,...,singlet_z_margin,time,tissue_type,tot_reads,nperts,ngenes,ncounts,percent_mito,percent_ribo,chembl-ID
AAACCTGAGACAAGCC,ACH-000174,True,0.114940,CAL62,normal,nan,thyroid cancer,µM,5.0,CAL62_THYROID,...,14.820933,24,cell_line,1094,1,3758,17155.0,4.447683,28.079277,CHEMBL443684
AAACCTGAGAGATGAG,ACH-000601,True,0.122660,MIAPACA2,normal,nan,pancreatic cancer,µM,5.0,MIAPACA2_PANCREAS,...,14.746431,24,cell_line,1009,1,4009,19764.0,7.468124,25.460433,CHEMBL443684
AAACCTGAGCGTTGCC,ACH-000601,True,0.176608,MIAPACA2,normal,nan,pancreatic cancer,µM,5.0,MIAPACA2_PANCREAS,...,16.011316,24,cell_line,3022,1,5772,47352.0,2.755955,27.965028,CHEMBL443684


In [6]:
# Calculate an embedding because CELLxGENE requires one
sc.tl.pca(adata)

## Curator non-perturbation data

In [7]:
adata.obs["disease"] = adata.obs["disease"].cat.rename_categories(
    {
        "colon/colorectal cancer": "colorectal cancer",
        "rhabdoid": "rhabdoid tumor",
        "bladder cancer": "urinary bladder carcinoma",
        "endometrial/uterine cancer": "uterine corpus cancer",
    }
)

adata.obs["cell_line"] = bt.CellLine.public(
    source=bt.Source.filter(name="depmap").one()
).standardize(adata.obs["cell_line"], field="name")

adata.obs["cell_line"] = adata.obs["cell_line"].astype('category')

# JQ1 is always uppercased
adata.obs["perturbation"] = adata.obs["perturbation"].cat.rename_categories(
    lambda category: category if category == "JQ1" else category.lower()
)

adata.obs = adata.obs.rename(columns={"DepMap_ID": "depmap_id"})
adata.obs = adata.obs.drop(columns="percent.mito")

In [8]:
curator = pt.dt.PerturbationCurator(
    adata, using_key=THIS_INSTANCE
)
curator.validate()

→ added defaults to the AnnData object: {'assay': 'unknown', 'cell_type': 'unknown', 'development_stage': 'unknown', 'donor_id': 'unknown', 'self_reported_ethnicity': 'unknown', 'suspension_type': 'cell', 'genetic_treatments': '', 'compound_treatments': '', 'environmental_treatments': '', 'combination_treatments': ''}
→ validating metadata using registries of instance laminlabs/pertpy-datasets
• saving validated records of 'var_index'


IntegrityError: duplicate key value violates unique constraint "lnschema_bionty_gene_uid_key"
DETAIL:  Key (uid)=(3BEZYDTfG7t4) already exists.


In [23]:
bt.Tissue.filter().all().delete()

In [8]:
adata.obs["sex"] = adata.obs["sex"].replace({"Unknown": "unknown"})

In [9]:
efo_lo = bt.ExperimentalFactor.public().lookup()

In [10]:
adata.obs["assay"] = efo_lo.single_cell_rna_sequencing.name

In [11]:
adata = adata[:, ~adata.var_names.isin(curator.non_validated["var_index"])].copy()

In [12]:
# Need to recreate Curator object because we are using a new object
curator = pt.dt.PerturbationCurator(adata, using_key=THIS_INSTANCE)
curator.validate()

→ validating metadata using registries of instance test-perturbation
• saving validated records of 'var_index'
• saving validated terms of 'assay'
• saving validated terms of 'cell_type'
• saving validated terms of 'development_stage'
• saving validated terms of 'disease'
• saving validated terms of 'donor_id'
• saving validated terms of 'self_reported_ethnicity'
• saving validated terms of 'sex'
• saving validated terms of 'suspension_type'
• saving validated terms of 'tissue_type'
• saving validated terms of 'organism'
• saving validated terms of 'cell_line'
• saving validated terms of 'genetic_treatments'
• saving validated terms of 'compound_treatments'
• saving validated terms of 'environmental_treatments'
• saving validated terms of 'combination_treatments'
✓ var_index is validated against Gene.ensembl_gene_id
✓ assay is validated against ExperimentalFactor.name
• mapping cell_type on CellType.name
!    1 terms is not validated: 'unknown'
→ fix typos, remove non-existent values, 

False

In [13]:
curator.add_new_from("all")

• saving validated records of 'var_index'
• saving validated terms of 'assay'
• saving validated terms of 'cell_type'
✓ added 1 record with CellType.name for cell_type: 'unknown'
• saving validated terms of 'development_stage'
✓ added 1 record with DevelopmentalStage.name for development_stage: 'unknown'
• saving validated terms of 'disease'
• saving validated terms of 'donor_id'
✓ added 1 record with ULabel.name for donor_id: 'unknown'
• saving validated terms of 'self_reported_ethnicity'
✓ added 1 record with Ethnicity.name for self_reported_ethnicity: 'unknown'
• saving validated terms of 'sex'
✓ added 1 record with Phenotype.name for sex: 'unknown'
• saving validated terms of 'suspension_type'
✓ added 1 record with ULabel.name for suspension_type: 'cell'
• saving validated terms of 'tissue_type'
✓ added 1 record with ULabel.name for tissue_type: 'cell_line'
• saving validated terms of 'organism'
• saving validated terms of 'cell_line'
• saving validated terms of 'genetic_treatments

In [14]:
curator.validate()

→ validating metadata using registries of instance test-perturbation
• saving validated records of 'var_index'
• saving validated terms of 'assay'
• saving validated terms of 'cell_type'
• saving validated terms of 'development_stage'
• saving validated terms of 'disease'
• saving validated terms of 'donor_id'
• saving validated terms of 'self_reported_ethnicity'
• saving validated terms of 'sex'
• saving validated terms of 'suspension_type'
• saving validated terms of 'tissue_type'
• saving validated terms of 'organism'
• saving validated terms of 'cell_line'
• saving validated terms of 'genetic_treatments'
• saving validated terms of 'compound_treatments'
• saving validated terms of 'environmental_treatments'
• saving validated terms of 'combination_treatments'
✓ var_index is validated against Gene.ensembl_gene_id
✓ assay is validated against ExperimentalFactor.name
✓ cell_type is validated against CellType.name
✓ development_stage is validated against DevelopmentalStage.name
✓ disea

True

All treatment columns validate but that's only because they're all empty.

## Curate perturbations

In [15]:
# Move
adata.obs["genetic_treatments"] = adata.obs["perturbation"].where(
    adata.obs["perturbation_type"] == "CRISPR", None
)
adata.obs["compound_treatments"] = adata.obs["perturbation"].where(
    adata.obs["perturbation_type"] == "drug", None
)

### Genetic treatments

In [16]:
list(adata.obs["genetic_treatments"].unique())

[nan, 'sggpx4-2', 'sglacz', 'sggpx4-1', 'sgor2j2']

In [17]:
treatments = [
    ("sggpx4-1", "GPX4", "Glutathione Peroxidase 4"),
    ("sggpx4-2", "GPX4", "Glutathione Peroxidase 4"),
    ("sgor2j2", "or2j2", "Olfactory receptor family 2 subfamily J member 2"),
    ("sglacz", "lacz", "beta-galactosidase control"),  # Control from E. coli
]
organism = bt.Organism.lookup().human

genetic_treatments = []
for name, symbol, target_name in treatments:
    treatment = wl.GeneticTreatment(system="CRISPR KO", name=name).save()
    if symbol != "lacz":
        gene_result = bt.Gene.from_source(symbol=symbol, organism=organism)
        gene = gene_result[0] if isinstance(gene_result, list) else gene_result
        gene = gene.save()
    else:
        gene = bt.Gene(symbol=symbol, organism=organism).save()
    target = wl.TreatmentTarget(name=target_name).save()
    target.genes.add(gene)
    treatment.targets.add(target)
    genetic_treatments.append(treatment)

✓ created 1 Gene record from Bionty matching symbol: 'GPX4'
! record with similar name exists! did you mean to load it?


,uid,name,system,sequence,on_target_score,off_target_score,run_id,created_at,created_by_id
id,,,,,,,,,
1,PuVY1Nkbqahe,sggpx4-1,CRISPR KO,None,None,None,None,2024-10-29 14:07:23.517967+00:00,1


→ returning existing TreatmentTarget record with same name: 'Glutathione Peroxidase 4'
✓ created 1 Gene record from Bionty matching synonyms: 'or2j2'
! ambiguous validation in Bionty for 1 record: 'OR2J2'


In [18]:
curator.validate()

→ validating metadata using registries of instance test-perturbation
• saving validated records of 'var_index'
• saving validated terms of 'assay'
• saving validated terms of 'cell_type'
• saving validated terms of 'development_stage'
• saving validated terms of 'disease'
• saving validated terms of 'donor_id'
• saving validated terms of 'self_reported_ethnicity'
• saving validated terms of 'sex'
• saving validated terms of 'suspension_type'
• saving validated terms of 'tissue_type'
• saving validated terms of 'organism'
• saving validated terms of 'cell_line'
• saving validated terms of 'genetic_treatments'
• saving validated terms of 'compound_treatments'
• saving validated terms of 'environmental_treatments'
• saving validated terms of 'combination_treatments'
✓ var_index is validated against Gene.ensembl_gene_id
✓ assay is validated against ExperimentalFactor.name
✓ cell_type is validated against CellType.name
✓ development_stage is validated against DevelopmentalStage.name
✓ disea

False

### Compounds

In [19]:
compounds = wl.Compound.from_values(adata.obs["compound_treatments"], field="name")

✓ created 8 Compound records from Bionty matching name: 'trametinib', 'afatinib', 'dabrafenib', 'gemcitabine', 'navitoclax', 'bortezomib', 'JQ1', 'everolimus'
! did not create Compound records for 6 non-validated names: 'azd5591', 'brd3379', 'control', 'idasanutlin', 'prexasertib', 'taselisib'


In [20]:
# The remaining compounds are not in chebi and we create records for them
for missing in [
    "azd5591",
    "brd3379",
    "control",
    "idasanutlin",
    "prexasertib",
    "taselisib",
]:
    compounds.append(wl.Compound(name=missing))
ln.save(compounds)

In [21]:
drug_metadata = adata.obs[adata.obs["compound_treatments"].notna()]

unique_treatments = drug_metadata[
    ["perturbation", "dose_unit", "dose_value"]
].drop_duplicates()

compound_treatments = []
for _, row in unique_treatments.iterrows():
    compound = wl.Compound.get(name=row["perturbation"])
    treatment = wl.CompoundTreatment(
        name=compound.name,
        concentration=row["dose_value"],
        concentration_unit=row["dose_unit"],
    )
    compound_treatments.append(treatment)

ln.save(compound_treatments)

In [22]:
compounds_to_targets = {
    "trametinib": ("MAPK/ERK pathway", ["P36507"]),
    "afatinib": ("EGFR, HER2, HER4 signaling", ["P00533", "Q9UK79", "Q15303"]),
    "dabrafenib": ("MAPK/ERK pathway", ["P15056"]),
    "gemcitabine": ("DNA synthesis inhibition", ["P23921"]),  # No single protein target
    "navitoclax": ("Apoptosis regulation", ["P10415", "Q07812"]),
    "bortezomib": ("Proteasome pathway", ["P49721"]),
    "brd3379": ("Transcription regulation (BET proteins)", ["O60885"]),
    "JQ1": ("Transcription regulation (BET proteins)", ["O60885"]),
    "azd5591": ("Apoptosis regulation", ["Q07820"]),
    "control": ("Baseline", [None]),  # No target for control
    "prexasertib": ("DNA damage response", ["O14757"]),
    "taselisib": ("PI3K/AKT/mTOR pathway", ["P42336", "O00329", "P48736"]),
    "idasanutlin": ("p53 regulation", ["Q00987"]),
    "everolimus": ("mTOR pathway", ["P42345"])
}


for compound_treatment_name, targets_tuple in compounds_to_targets.items():
    compound_treatment = wl.CompoundTreatment.get(name=compound_treatment_name)
    target = wl.TreatmentTarget(name=targets_tuple[0]).save()
    proteins = []
    for id in targets_tuple[1]:
        if id is not None:
            proteins.append(bt.Protein.from_source(uniprotkb_id=id).save())
    target.proteins.set(proteins)
    compound_treatment.targets.add(target)

✓ created 1 Protein record from Bionty matching uniprotkb_id: 'P36507'
✓ created 1 Protein record from Bionty matching uniprotkb_id: 'P00533'
✓ created 1 Protein record from Bionty matching uniprotkb_id: 'Q9UK79'
✓ created 1 Protein record from Bionty matching uniprotkb_id: 'Q15303'
→ returning existing TreatmentTarget record with same name: 'MAPK/ERK pathway'
✓ created 1 Protein record from Bionty matching uniprotkb_id: 'P15056'
✓ created 1 Protein record from Bionty matching uniprotkb_id: 'P23921'
✓ created 1 Protein record from Bionty matching uniprotkb_id: 'P10415'
✓ created 1 Protein record from Bionty matching uniprotkb_id: 'Q07812'
! record with similar name exists! did you mean to load it?


,uid,name,description,run_id,created_at,created_by_id
id,,,,,,
4,CjakOwn0,MAPK/ERK pathway,None,None,2024-10-29 14:07:36.548584+00:00,1


✓ created 1 Protein record from Bionty matching uniprotkb_id: 'P49721'
! record with similar name exists! did you mean to load it?


,uid,name,description,run_id,created_at,created_by_id
id,,,,,,
7,5EdhKMoj,Apoptosis regulation,None,None,2024-10-29 14:07:46.070632+00:00,1


✓ created 1 Protein record from Bionty matching uniprotkb_id: 'O60885'
→ returning existing TreatmentTarget record with same name: 'Transcription regulation (BET proteins)'
→ returning existing TreatmentTarget record with same name: 'Apoptosis regulation'
✓ created 1 Protein record from Bionty matching uniprotkb_id: 'Q07820'
✓ created 1 Protein record from Bionty matching uniprotkb_id: 'O14757'
! records with similar names exist! did you mean to load one of them?


...

✓ created 1 Protein record from Bionty matching uniprotkb_id: 'P42336'
✓ created 1 Protein record from Bionty matching uniprotkb_id: 'O00329'
✓ created 1 Protein record from Bionty matching uniprotkb_id: 'P48736'
! records with similar names exist! did you mean to load one of them?


,uid,name,description,run_id,created_at,created_by_id
id,,,,,,
7,5EdhKMoj,Apoptosis regulation,None,None,2024-10-29 14:07:46.070632+00:00,1
9,Ppl50wZN,Transcription regulation (BET proteins),None,None,2024-10-29 14:07:50.636207+00:00,1


✓ created 1 Protein record from Bionty matching uniprotkb_id: 'Q00987'
! records with similar names exist! did you mean to load one of them?


,uid,name,description,run_id,created_at,created_by_id
id,,,,,,
12,2HxVNgiL,PI3K/AKT/mTOR pathway,None,None,2024-10-29 14:07:54.820080+00:00,1
4,CjakOwn0,MAPK/ERK pathway,None,None,2024-10-29 14:07:36.548584+00:00,1
8,yBJ85bnI,Proteasome pathway,None,None,2024-10-29 14:07:49.114596+00:00,1


✓ created 1 Protein record from Bionty matching uniprotkb_id: 'P42345'


## References

In [23]:
reference = fr.Reference(
    name="Multiplexed single-cell transcriptional response profiling to define cancer vulnerabilities and therapeutic mechanism of action",
    abbr="McFarland 2020",
    url="https://www.nature.com/articles/s41467-020-17440-w",
    doi="10.1038/s41467-020-17440-w",
    text=(
        "Assays to study cancer cell responses to pharmacologic or genetic perturbations are typically "
        "restricted to using simple phenotypic readouts such as proliferation rate. Information-rich assays, "
        "such as gene-expression profiling, have generally not permitted efficient profiling of a given "
        "perturbation across multiple cellular contexts. Here, we develop MIX-Seq, a method for multiplexed "
        "transcriptional profiling of post-perturbation responses across a mixture of samples with single-cell "
        "resolution, using SNP-based computational demultiplexing of single-cell RNA-sequencing data. We show "
        "that MIX-Seq can be used to profile responses to chemical or genetic perturbations across pools of 100 "
        "or more cancer cell lines. We combine it with Cell Hashing to further multiplex additional experimental "
        "conditions, such as post-treatment time points or drug doses. Analyzing the high-content readout of "
        "scRNA-seq reveals both shared and context-specific transcriptional response components that can identify "
        "drug mechanism of action and enable prediction of long-term cell viability from short-term transcriptional "
        "responses to treatment."
    ),
).save()

## Remove unused columns

In [24]:
adata.obs = adata.obs.drop(
    [
        "depmap_id",
        "cancer",
        "cell_quality",
        "channel",
        "perturbation",
        "perturbation_type",
        "singlet_dev",
        "singlet_dev_z",
        "singlet_margin",
        "singlet_z_margin",
        "nperts",
        "ngenes",
        "ncounts",
        "cell_det_rate",
        "doublet_GMM_prob",
        "doublet_dev_imp",
        "doublet_z_margin",
        'doublet_CL1',
        'doublet_CL2'
    ],
    axis=1,
)

## Register curated artifact

In [25]:
artifact = curator.save_artifact(description="McFarland AnnData")

→ validating metadata using registries of instance test-perturbation
• saving validated records of 'var_index'
• saving validated terms of 'assay'
• saving validated terms of 'cell_type'
• saving validated terms of 'development_stage'
• saving validated terms of 'disease'
• saving validated terms of 'donor_id'
• saving validated terms of 'self_reported_ethnicity'
• saving validated terms of 'sex'
• saving validated terms of 'suspension_type'
• saving validated terms of 'tissue_type'
• saving validated terms of 'organism'
• saving validated terms of 'cell_line'
• saving validated terms of 'genetic_treatments'
• saving validated terms of 'compound_treatments'
• saving validated terms of 'environmental_treatments'
• saving validated terms of 'combination_treatments'
✓ var_index is validated against Gene.ensembl_gene_id
✓ assay is validated against ExperimentalFactor.name
✓ cell_type is validated against CellType.name
✓ development_stage is validated against DevelopmentalStage.name
✓ disea

In [26]:
# Set the perturbations and references
artifact.genetic_treatments.set(genetic_treatments)
artifact.compound_treatments.set(compound_treatments)
artifact.references.add(reference)

In [27]:
artifact.describe()

Artifact(uid='y3B2kwItyAgpru4G0000', is_latest=True, description='McFarland AnnData', suffix='.h5ad', type='dataset', size=3345456, hash='dETFEMNWe6nH2enh1PNUTg', n_observations=1000, _hash_type='md5', _accessor='AnnData', visibility=1, _key_is_virtual=True, created_at=2024-10-29 14:08:04 UTC)
  Provenance
    .storage = '/home/zeth/PycharmProjects/pertpy-datasets/lamin/test-perturbation'
    .created_by = 'zethson'
  Labels
    .references = 'Multiplexed single-cell transcriptional response profiling to define cancer vulnerabilities and therapeutic mechanism of action'
    .genetic_treatments = 'sggpx4-1', 'sggpx4-2', 'sgor2j2', 'sglacz'
    .compound_treatments = 'trametinib', 'afatinib', 'dabrafenib', 'gemcitabine', 'navitoclax', 'bortezomib', 'brd3379', 'JQ1', 'azd5591', 'control', ...
    .organisms = 'human'
    .cell_types = 'unknown'
    .diseases = 'lung cancer', 'urinary bladder carcinoma', 'colorectal cancer', 'head and neck cancer', 'brain cancer', 'skin cancer', 'gastric c

In [31]:
adata.obs

,cell_line,disease,dose_unit,dose_value,hash_assignment,hash_tag,num_SNPs,organism,sex,singlet_ID,...,assay,cell_type,development_stage,donor_id,self_reported_ethnicity,suspension_type,genetic_treatments,compound_treatments,environmental_treatments,combination_treatments
AACTGGTGTCTCTCTG,LUDLU-1,lung cancer,µM,0.1,nan,nan,481,human,Male,LUDLU1_LUNG,...,single-cell RNA sequencing,unknown,unknown,unknown,unknown,cell,NaN,trametinib,,
ATAGGCTCAGATTTCG,LU99,lung cancer,µM,0.5,nan,nan,1003,human,Male,LU99_LUNG,...,single-cell RNA sequencing,unknown,unknown,unknown,unknown,cell,NaN,afatinib,,
GCCAAATCAAGCCGTC,J82,urinary bladder carcinoma,µM,0.1,nan,nan,647,human,Male,J82_URINARY_TRACT,...,single-cell RNA sequencing,unknown,unknown,unknown,unknown,cell,NaN,dabrafenib,,
CGGAGAAGTCGCGTCA,HCT-15,colorectal cancer,µM,0.1,nan,nan,30,human,Male,HCT15_LARGE_INTESTINE,...,single-cell RNA sequencing,unknown,unknown,unknown,unknown,cell,NaN,gemcitabine,,
TAGTTGGAGATCGATA,YD-10B,head and neck cancer,nan,NaN,nan,nan,874,human,Male,YD10B_UPPER_AERODIGESTIVE_TRACT,...,single-cell RNA sequencing,unknown,unknown,unknown,unknown,cell,sggpx4-2,NaN,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GATCTAGTCATCGGAT,NCI-H1581,lung cancer,µM,0.1,nan,nan,480,human,Male,NCIH1581_LUNG,...,single-cell RNA sequencing,unknown,unknown,unknown,unknown,cell,NaN,dabrafenib,,
AACTCTTAGTCCCACG,LS1034,colorectal cancer,µM,0.0,nan,nan,366,human,Male,LS1034_LARGE_INTESTINE,...,single-cell RNA sequencing,unknown,unknown,unknown,unknown,cell,NaN,control,,
CACTGTCCAGTCACGC,A549,lung cancer,µM,2.5,nan,nan,528,human,Male,A549_LUNG,...,single-cell RNA sequencing,unknown,unknown,unknown,unknown,cell,NaN,JQ1,,
CACCTTGGTCGACTAT,NCI-H2347,lung cancer,µM,0.0,nan,nan,1230,human,Female,NCIH2347_LUNG,...,single-cell RNA sequencing,unknown,unknown,unknown,unknown,cell,NaN,control,,
